Test dataloader output

In [1]:
from dataloader import REMISkylineToMidiVAEDataset

#### utils

In [2]:
import torch
import pickle
from scipy.spatial import distance

# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')

def numpy_to_tensor(arr, use_gpu=True, device='cuda:0'):
  if use_gpu:
    return torch.tensor(arr).to(device).float()
  else:
    return torch.tensor(arr).float()

def tensor_to_numpy(tensor):
  return tensor.cpu().detach().numpy()

def pickle_load(f):
  return pickle.load(open(f, 'rb'))

def pickle_dump(obj, f):
  pickle.dump(obj, open(f, 'wb'), protocol=pickle.HIGHEST_PROTOCOL)

#### initialize dataset

In [3]:
import yaml
config_path = "/home/yihsin/MidiStyleTransfer/MuseMorphose/config/skyline.yaml"
config = yaml.load(open(config_path, 'r'), Loader=yaml.FullLoader)

In [4]:
dset = REMISkylineToMidiVAEDataset(
    config['data']['data_dir'], config['data']['vocab_path'], 
    do_augment=True, use_composer_cls = False,
    model_enc_seqlen=config['data']['enc_seqlen'], 
    model_dec_seqlen=config['data']['dec_seqlen'], 
    model_max_bars=config['data']['max_bars'],
    pieces=pickle_load(config['data']['train_split']),
    pad_to_same=True
)

[preparing data] now at #0
[preparing data] now at #200
[preparing data] now at #400
[preparing data] now at #600
[preparing data] now at #800
[preparing data] now at #1000
[preparing data] now at #1200
[preparing data] now at #1400
[preparing data] now at #1600
[preparing data] now at #1800
[preparing data] now at #2000
[preparing data] now at #2200
[preparing data] now at #2400
[preparing data] now at #2600
[preparing data] now at #2800
[preparing data] now at #3000
[preparing data] now at #3200
[preparing data] now at #3400
[preparing data] now at #3600
[preparing data] now at #3800
[preparing data] now at #4000
[preparing data] now at #4200
[preparing data] now at #4400
[preparing data] now at #4600
[preparing data] now at #4800
[preparing data] now at #5000
[preparing data] now at #5200
[preparing data] now at #5400
[preparing data] now at #5600


In [5]:
from torch.utils.data import DataLoader
dloader = DataLoader(dset, batch_size=config['data']['batch_size'], shuffle=True, num_workers=8)

In [6]:
dset_val = REMISkylineToMidiVAEDataset(
    config['data']['data_dir'], config['data']['vocab_path'], 
    do_augment=False, use_composer_cls = False,
    model_enc_seqlen=config['data']['enc_seqlen'], 
    model_dec_seqlen=config['data']['dec_seqlen'], 
    model_max_bars=config['data']['max_bars'],
    pieces=pickle_load(config['data']['val_split']),
    pad_to_same=True
)
dloader_val = DataLoader(dset_val, batch_size=config['data']['batch_size'], shuffle=True, num_workers=8)

[preparing data] now at #0
[preparing data] now at #200
[preparing data] now at #400
[preparing data] now at #600
[preparing data] now at #800
[preparing data] now at #1000
[preparing data] now at #1200
[preparing data] now at #1400
[preparing data] now at #1600


#### initialize model

In [7]:
import sys
sys.path.append('./model')
from model.musemorphose import MuseMorphose

In [8]:
!export CUDA_VISIBLE_DEVICES=1
device = "cuda"

In [15]:
mconf = config['model']
model = MuseMorphose(
  mconf['enc_n_layer'], mconf['enc_n_head'], mconf['enc_d_model'], mconf['enc_d_ff'],
  mconf['dec_n_layer'], mconf['dec_n_head'], mconf['dec_d_model'], mconf['dec_d_ff'],
  mconf['d_latent'], mconf['d_embed'], dset.vocab_size,
  d_polyph_emb=mconf['d_polyph_emb'], d_rfreq_emb=mconf['d_rfreq_emb'],
  cond_mode=mconf['cond_mode'], use_attr_cls=False
).to(device)

/home/yihsin/miniforge3/envs/muse/lib/python3.10/site-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


#### start testing

In [ ]:
from torch import nn, optim
max_lr, min_lr = config['training']['max_lr'], config['training']['min_lr']

lr_decay_steps = config['training']['lr_decay_steps']
lr_warmup_steps = config['training']['lr_warmup_steps']
no_kl_steps = config['training']['no_kl_steps']
kl_cycle_steps = config['training']['kl_cycle_steps']
kl_max_beta = config['training']['kl_max_beta']
free_bit_lambda = config['training']['free_bit_lambda']
max_lr, min_lr = config['training']['max_lr'], config['training']['min_lr']

opt_params = filter(lambda p: p.requires_grad, model.parameters())
optimizer = optim.Adam(opt_params, lr=max_lr)
scheduler = optim.lr_scheduler.CosineAnnealingLR(
    optimizer, lr_decay_steps, eta_min=min_lr
  )

import time
epoch = 0
optim = optimizer
sched = scheduler

model.train()
print ('[epoch {:03d}] training ...'.format(epoch))
print ('[epoch {:03d}] # batches = {}'.format(epoch, len(dloader)))
st = time.time()

In [13]:
for batch_idx, batch_samples in enumerate(dloader):
        model.zero_grad()
        batch_enc_inp = batch_samples['enc_input'].permute(2, 0, 1).to(device)
        batch_dec_inp = batch_samples['dec_input'].permute(1, 0).to(device)
        batch_dec_tgt = batch_samples['dec_target'].permute(1, 0).to(device)
        batch_inp_bar_pos = batch_samples['bar_pos'].to(device)
        batch_inp_lens = batch_samples['length']
        batch_padding_mask = batch_samples['enc_padding_mask'].to(device)
        batch_composer_cls = batch_samples['composer_cls'].permute(1, 0).to(device)
        break

In [16]:
mu, logvar, dec_logits = model(
      batch_enc_inp, batch_dec_inp, 
      batch_inp_bar_pos, batch_composer_cls,
      padding_mask=batch_padding_mask
    )

In [18]:
kl_beta = kl_max_beta
losses = model.compute_loss(mu, logvar, kl_beta, free_bit_lambda, dec_logits, batch_dec_tgt)